# Assignment 2 specification

The purpose of this assignment is to use clustering and classification to predict various aspects of internet users based on data collected from a survey.

The survey has a large number of columns (features) so you will need to choose a suitable subset both for clustering and for classification.

The rest of this notebook provides basic help with preparing the data for analysis.

## Background

In [ ]:
import pandas as pd
import numpy as np

See https://www.openml.org/d/372 for description and https://www.openml.org/data/download/52407/internet_usage.arff for the data file itself. The data comprises mostly binary and some categorical (multi-valued) columns, with just 2 numeric columns, all relating to internet users circa 1997.

The first thing to do is to load the data.

In [ ]:
from scipy.io import arff
filePath = 'internet_usage.arff'
data, meta = arff.loadarff(filePath)
df = pd.DataFrame(data)
df.head()

As can be seen, the data is loaded into a dataframe but in a binary representation. We choose to convert the binary text into strings, as they are much easier to handle.

In [ ]:
for col in df.columns:
  df[col] = df[col].apply(lambda x: x.decode("utf-8"))
df.head()

The dataframe looks more standard now, but we notice that there is abn anonymised user code 'who' which is a candidate for the dataframe's index. We check that each row has a unique 'who' value:

In [ ]:
numRows = df.shape[0]
numUniq = len(df['who'].unique().tolist())
print(numRows-numUniq)

It does, so we set 'who' as the index and it no longer appears in the list of columns, which we can check below.

In [ ]:
if 'who' in df.columns:
  df.set_index('who', inplace=True)

print(df.columns)

As can be seen, we have ensured that the 'who' column is no longer available as a feature. Also note that all columns are treated as 'object', effectively as strings. For your convenience, I have classified the columns for you, see below. I have also changed the types of numeric and 'boolean' (0,1)-valued columns. The latter are then binarised and ready for analysis.

In [ ]:
numericCols = ['Age', 'Opinions_on_Censorship']
boolCols = ['Community_Membership_Family', 'Community_Membership_Hobbies',
       'Community_Membership_None', 'Community_Membership_Other',
       'Community_Membership_Political', 'Community_Membership_Professional',
       'Community_Membership_Religious', 'Community_Membership_Support',
       'Disability_Cognitive', 'Disability_Hearing',
       'Disability_Motor', 'Disability_Not_Impaired', 'Disability_Not_Say',
       'Disability_Vision', 'How_You_Heard_About_Survey_Banner',
       'How_You_Heard_About_Survey_Friend',
       'How_You_Heard_About_Survey_Mailing_List',
       'How_You_Heard_About_Survey_Others',
       'How_You_Heard_About_Survey_Printed_Media',
       'How_You_Heard_About_Survey_Remebered',
       'How_You_Heard_About_Survey_Search_Engine',
       'How_You_Heard_About_Survey_Usenet_News',
       'How_You_Heard_About_Survey_WWW_Page', 'Not_Purchasing_Bad_experience',
       'Not_Purchasing_Bad_press', 'Not_Purchasing_Cant_find',
       'Not_Purchasing_Company_policy', 'Not_Purchasing_Easier_locally',
       'Not_Purchasing_Enough_info', 'Not_Purchasing_Judge_quality',
       'Not_Purchasing_Never_tried', 'Not_Purchasing_No_credit',
       'Not_Purchasing_Not_applicable', 'Not_Purchasing_Not_option',
       'Not_Purchasing_Other', 'Not_Purchasing_Prefer_people',
       'Not_Purchasing_Privacy', 'Not_Purchasing_Receipt',
       'Not_Purchasing_Security', 'Not_Purchasing_Too_complicated',
       'Not_Purchasing_Uncomfortable', 'Not_Purchasing_Unfamiliar_vendor',
           'Who_Pays_for_Access_Dont_Know',
       'Who_Pays_for_Access_Other', 'Who_Pays_for_Access_Parents',
       'Who_Pays_for_Access_School', 'Who_Pays_for_Access_Self',
       'Who_Pays_for_Access_Work']
strCols = ['Actual_Time', 'Community_Building', 'Country', 'Education_Attainment', 'Falsification_of_Information',
           'Gender', 'Household_Income', 'Major_Geographical_Location', 'Major_Occupation', 'Marital_Status',
           'Most_Import_Issue_Facing_the_Internet', 'Primary_Computing_Platform', 'Primary_Language',
           'Primary_Place_of_WWW_Access', 'Race', 'Registered_to_Vote', 'Sexual_Preference', 'Web_Ordering',
           'Web_Page_Creation', 'Willingness_to_Pay_Fees', 'Years_on_Internet' ]


for col in numericCols:
  df[col] = pd.to_numeric(df[col], errors='coerce')

for col in boolCols:
  df[col] = df[col].map({'0': 0, '1': 1})


According to the data description, the original internet_usage data had 2699 missing values in the 'Primary_Computing_Platform' column. In this version of the dataset, the missing values have already been replaced with '?', see below, so no further action is needed.

In [ ]:
col = 'Primary_Computing_Platform'
df[col].value_counts()

Most scikit-learn clustering and classification algorithms require _all_ features to be numeric. The `boolCols` features are already encoded as numeric 0,1 but the `strCols` need further processing. The code below shows how to _One Hot Encode_ a selection of features, and how to combine them in a dataframe. Note that we create a Python `dict` of `OneHotEncoder`s rather than reusing a single instance, so that we have the option of transforming back to the original string labels later.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import random

ohe = dict()

# Choose a seed so that this code is repeatable, and select some features for the model 
random.seed(42)
originalStrCols = random.sample(strCols,k=4)
print(originalStrCols)
sampledBoolCols = random.sample(boolCols,k=5)
print(sampledBoolCols)

# Create an empty dataframe
dfSub = pd.DataFrame()

for col in originalStrCols:
  ohe[col] = OneHotEncoder(sparse=False)
  X = ohe[col].fit_transform(df[col].values.reshape(-1,1))
  # See https://stackoverflow.com/a/4843172
  dfOneHot = pd.DataFrame(X, columns = [col+'-'+str(int(i)) for i in range(X.shape[1])])
  dfSub = pd.concat([dfSub, dfOneHot], axis=1)

# Assign the index so that it matches that of the original df
dfSub.set_axis(df.index, axis='index', inplace=True)

# Add in the sampledBoolcols
dfSub = pd.concat([dfSub, df[sampledBoolCols]], axis=1)

# The following is the matrix of samples x features
dfSub.head(10)

In [ ]:
# Add in the predicted column.

# Depending on the algorithm used, this might also need to be binarized.
# By convention, we use the LabelBinarizer for this.
binarizeLabel = True
y = df['Major_Occupation']
if binarizeLabel:
  from sklearn import preprocessing
  lbY = preprocessing.LabelBinarizer(sparse_output=False)

  # For the (string) label to predict, we use the 'Major_Occupation' column
  yBin = lbY.fit_transform(y)
  dfY = pd.DataFrame(data=yBin, index=df.index, columns=lbY.classes_)
  dfSub = pd.concat([dfSub, dfY], axis=1)
else:
  dfSub = pd.concat([dfSub, y], axis=1)

# The following matrix contains features and y
dfSub.head(10)

Lastly, if you wish to transform the one-hot-encoded features back to their original form, you can use something like:

In [ ]:
dx = dict()
for col in originalStrCols:
  derivedCol = [s for s in dfSub.columns if col+'-' in s]
  dx[col] = ohe[col].inverse_transform(dfSub[derivedCol])
print(dx)

## Your Tasks

### Clustering

1. Reviewing the data and its data dictionary, choose candidate feature subsets (with a maximum of 10 features per subset) that might be used to cluster the internet usage data described above. Justify your choice of feature subsets.
2. For each candidate feature subset, use hierarchical clustering, k-means, GMM and DBSCAN algorithms on it to identify possible groups of internet users.
3. Hence, discuss the advantages and disadvantages of the different clustering techniques for this task.
4. For bonus marks: Which combinations of features appear to perform best and why?

###  Classification

1. For prediction, you should use 'Major_Occupation' and 'Education_Attainment', the same candidate feature subsets, and at least 3 of the classification techniques that were taught in class:
   a. For each technique, find suitable configuration parameters (specific to that classification technique) by splitting the data into training and test sets and finding the settings that maximise the prediction accuracy and related scores.
   b. Given the best settings for each technique, use 10-fold cross-validation on each technique to rank the algorithms in terms of their performance on a given combination of features and predicted variable.
   c. Comment on the performance differences when predicting 'Major_Occupation' vs 'Education_Attainment'
3. Repeat this exercise but using the reduced dimension data from PCA. Comment on any differences you find. 

__Important__

Your attempt should be submitted as a Jupyter notebook. Use plenty of comments to explain your reasoning and to discuss what you find. Output should be presented inline using formatted print statements and/or plots.

# Clustering: a possible approach

1. Choose a set of features (columns), probably more than 10 (to have rich enough data) and less than 20 (to make it easier to work with)
2. The features should not "overlap" (if two or more have the same data, just choose one of them)
3. The features should capture "interesting" aspects of internet users and usage - that is where your insight/creativity is needed!
4. 10 or more features might be difficult to visualise/interpret, so it might be worth using a dimensionality reduction technique like PCA (see the Practical where it was applied to the MNIST digits data to reduce the dimensions from 64 to 41).
5. A selection of One-Hot-Encoded features should reduce well, say from 15 to 5 transformed dimensions (these numbers are just used as a guide!)
6. You can apply hierarchical clustering to either the original features or the transformed features.
7. You are looking for nicely distributed clusters in the dendrogram. You can play around with linkage and other choices but if you are still unsatisfied, it might be time to try a different set of features. You can change the level of dimensionality reduction (if used), or even add/remove candidate features, and try again.
8. There is no such thing as perfect data for clustering, so after a few tries, pick the best combination of features and suggest some partitional clustering algorithms and their settings.
9. If you can try several clustering schemes, so much the better - you can then see whether particular users cluster together consistently, regardless of the algorithm used to search for such clusters.
10. Using the cluster memberships, try to interpret what the clusters might be, e.g., one cluster might be "experienced internet users, using the internet mostly for work, based in the US".
11. If you transformed the features, you will definitely find it easier to interpret the clusters based on the original features, so you would just need to add the cluster membership labels (derived from the transformed features) row-by-row to the dataframe with the original feature subset.
12. Be sure to discuss and justify your findings.

# Classification: a possible approach

1. Again, it is advisable to select a subset of attributes, this time with a view to predicting `Major_Occupaion` and `Education_Attainment`, so they are non-redundant and "interesting".
2. You will need to choose 3 classification techniques and apply them to the original and transformed dimensions and the two labeled "response variables".
3. Since you are interested in comparing the classifiers and not just maximising the prediction accuracy for a specific classifier I recommend you use cross-validation to estimate the prediction accuracy _and_ its uncertainty.
4. You then need to discuss the performance of your classifiers, both in terms of sensitivitiy to settings of their own parameters, and how they compare to their peers.